# Nueva sección

In [1]:
from google.colab import files

In [2]:
import pandas as pd

In [ ]:
from pandas.io.json import json_normalize

In [ ]:
import json

Empezamos a abrir los archivos Json

In [ ]:

import ast
reviews = []
with open('/content/drive/MyDrive/Henry Final_1/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_reviews = pd.DataFrame(reviews)

In [ ]:
items = []
with open('/content/drive/MyDrive/Henry Final_1/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_items = pd.DataFrame(items)

In [ ]:
import json

# Lista para almacenar los objetos JSON individuales
user_games = []

# Abre el archivo JSON y carga su contenido en un diccionario
with open('/content/drive/MyDrive/Henry Final_1/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        try:
            json_data = json.loads(line)
            user_games.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en línea: {e}")

# Crear el DataFrame anidado
df_games = pd.DataFrame(user_games)

In [ ]:
df_games

Dado que el archivo de games no está anidado como json sino como texto el manejo es diferente.

Como no podemos desanidar el texto usamos explode que nos deje la primera observación de la serie.

In [ ]:
games= df_games.explode('genres')
games

In [ ]:
games= games.explode('specs')

In [ ]:
games

Quitamos los espacios vacíos y los nulos los cambiamos por Unknown.

In [ ]:
games = games.dropna(subset=['specs'])

In [ ]:
games = games.fillna({'specs': 'Unknown'})

In [ ]:
games = games.dropna(subset=['genres'])

In [ ]:
games = games.fillna({'genres': 'Unknown'})

In [ ]:
games

Empezamos a desanidar los archivos, dado que estos son un json anidado usamos una manera diferente a la usada en games

In [ ]:
reviews_desanidado = json_normalize(df_reviews.to_dict(orient='records'), 'reviews', meta=['user_id', 'user_url'])

In [ ]:
reviews_desanidado

In [ ]:
items_desanidado = json_normalize(df_items.to_dict(orient='records'), 'items', meta=['user_id','items_count','steam_id','user_url'])

In [ ]:
items_desanidado

Realizamos un merge para tener todos los datos en una sola base de datos.

In [ ]:
df_merged = reviews_desanidado.merge(items_desanidado, on=['user_id', 'user_url','item_id'])

In [ ]:
df_merged

In [ ]:
df_merged = df_merged.rename(columns={'item_id': 'id'})


In [ ]:
df_final = pd.merge(df_merged, games,on='id')

In [ ]:
df_final


Eliminamos todas las columnas que no creo conveniente usar.

In [ ]:

df_final = df_final.drop(columns=['app_name','early_access','tags','funny','posted', 'last_edited', 'helpful', 'item_name', 'playtime_2weeks','publisher','reviews_url','developer'])


In [ ]:
df_final

A continuación realizamos los cambios que creemos necesarios, como el manejo de los nulos y el tipo de variables

In [ ]:
df_final['recommend'] = df_final['recommend'].astype(int)

In [ ]:
df_final['price'] = df_final['price'].replace('Free To Play', 0)


In [ ]:
df_final['price'] = pd.to_numeric(df_final['price'], errors='coerce')

In [ ]:
df_final['price'] = df_final['price'].astype(float)

In [ ]:
df_final['price'] = df_final['price'].fillna(0)

In [ ]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'])
df_final['release_year'] = df_final['release_date'].dt.year



In [ ]:
df_final['release_year'] = df_final['release_year'].fillna(0)


In [ ]:
df_final['release_year'] = df_final['release_year'].astype(int)

In [ ]:
from datetime import datetime

In [ ]:
df_final.dtypes

In [ ]:
df_final.isnull().sum()

Por último convertimos nuestro data frame a un archivo pickel porque es más fiel a la conversión de datos y no tendremos errores como sucede con un csv o un pickel.

In [ ]:
df_final.to_pickle('df_final.pkl')

In [ ]:
df_final.to_csv('df_final.csv')

In [ ]:
df_final